In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import math
from torch.utils.data import Dataset, DataLoader
from sklearn import datasets
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.preprocessing import StandardScaler#para escalar caracteristicas
from sklearn.model_selection import train_test_split # separar mas facil la data de train y test
import pandas as pd
from sklearn.utils import shuffle
from ast import literal_eval

In [2]:
def PaddingSameSize(matrix,padd_type='constant',constant_value=0):
    length = np.array([len(matrix[i]) for i in range(len(matrix))])
    width = length.max()
    return_list=[]
    for i in range(len(matrix)):
        if len(matrix[i]) != width:
            if padd_type == 'constant':
                padd = np.pad(matrix[i], (0,width-len(matrix[i])), 'constant',constant_values = 0)
            else:
                padd = np.pad(matrix[i], (0,width-len(matrix[i])), padd_type)
        else:
            padd = matrix[i]
        return_list.append(padd)
    return_list = np.array(return_list)
    return return_list

In [3]:
def MatrixFormat_To_Vector(df_matrix_format):
    df_matrix_format = df_matrix_format.apply(lambda x: np.asarray(literal_eval(x)).astype(np.float32)) 
    max_x,max_y = {"index":0,"value":0},{"index":0,"value":0}
    vector_list = []
    vector_list_pad = []
    max_len = 0
    for i, matrix in df_matrix_format.items():
    #    if max_x['value'] < matrix.shape[0]:
    #        max_x['value'] = matrix.shape[0]
    #        max_x['index'] = i
    #    if max_y['value'] < matrix.shape[1]:
    #        max_y['value'] = matrix.shape[1]
    #        max_y['index'] = i
        vector = np.reshape(matrix, -1)
        vector_list.append(vector)

    #for v in vector_list:
    #    if max_len < len(v):
    #        max_len = len(v)
    #print("max len", max_len)
    vector_list_pad = PaddingSameSize(vector_list, 'constant')      
    
    #for i in vector_list_pad:
    #    print(i.shape)
    #    print(i)
    
    return vector_list_pad

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Esto garantiza que se ejecutara en GPU si esta disponible
print(device)

cpu


In [5]:
chars = ['a','b','c','d','e']
nums = ['1','2','c','3','e']
a = ['XD']

c = chars + nums + a
print(c)

['a', 'b', 'c', 'd', 'e', '1', '2', 'c', '3', 'e', 'XD']


In [6]:
df_raw = pd.read_csv('D:/.Memoria/Test/Jupyter/test_example.csv')
columns = list(df_raw.columns)
print(columns)
#  Eliminar los con tiempos de ejecución 0 o muy altos
df_clean = shuffle(df_raw[(df_raw['ql_rt_msec'] > 0) & (df_raw['ql_rt_msec'] < 1e4)])
df_clean.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean.csv',index=False)

### HAY QUE CONVERTIR ALGUNAS COLUMNAS Q ESTAN EN PORCENTAJE A FLOAT
df_clean['ql_rt_clocks'] = df_clean['ql_rt_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_seg'] = df_clean['ql_same_seg'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_same_page'] = df_clean['ql_same_page'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_cl_wait_clocks'] = df_clean['ql_cl_wait_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_clocks'] = df_clean['ql_c_clocks'].apply(lambda x: float(x.strip('%'))/100)
df_clean['ql_c_cl_wait'] = df_clean['ql_c_cl_wait'].apply(lambda x: float(x.strip('%'))/100)

### Variable con las matrices




print(f'shape df_raw: {df_raw.shape}')
print(f'shape df_clean: {df_clean.shape}')



x = MatrixFormat_To_Vector(df_clean['matrix_format'])
x_trans = x.T
new_features = []
print(f'shape x: {x.shape}')
print(f'shape x_trans: {x_trans.shape}')
for i in range(x_trans.shape[0]):
    new_features.append('var_'+str(i))
    df_clean['var_'+str(i)] = x_trans[i]
    if i % 1000 == 0:
        print("Sigo vivo xd")

print(f'shape df_clean + caracteristicas de vectores: {df_clean.shape}')
        
msk = np.random.rand(len(df_clean)) <= 0.8
df_train = df_clean[msk]
df_test = df_clean[~msk]

df_train.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_train.csv',index=False)
df_test.to_csv('D:/.Memoria/Test/Jupyter/test_example_clean_test.csv',index=False)

#print(f'shape df_train: {df_train.shape}')
#print(f'shape df_test: {df_test.shape}')

normal_features = [
    'limit',
    'ql_rt_clocks',
    'ql_rnd_rows',
    'ql_seq_rows',
    'ql_same_seg',
    'ql_same_page',
    'ql_disk_reads',
    'ql_spec_disk_reads',
    'ql_cl_wait_clocks',
    'ql_c_msec',
    'ql_c_disk',
    'ql_c_clocks',
    'ql_cl_messages',
    'ql_c_cl_wait'
] 
all_features_to_evaluate = normal_features + new_features

## FEATURES - TRAIN Y TEST
X_df_train = df_train[all_features_to_evaluate]
X_df_test = df_test[all_features_to_evaluate]

X_numpy_train = X_df_train.to_numpy().astype(np.float32)
X_numpy_test = X_df_test.to_numpy().astype(np.float32)


## TARGETS - TRAIN Y TEST
y_df_train = df_train['ql_rt_msec']
y_df_test = df_test['ql_rt_msec']

y_numpy_train = y_df_train.to_numpy().astype(np.float32)
y_numpy_test = y_df_test.to_numpy().astype(np.float32)

# Pasarlos a Torch. 
X_train = torch.from_numpy(X_numpy_train)
X_test = torch.from_numpy(X_numpy_test)
y_train = torch.from_numpy(y_numpy_train)
y_test = torch.from_numpy(y_numpy_test)
# También pasar los targets de vector fila a vector columna
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)


print("-----------------------")
print(f'shape X_numpy_train: {X_numpy_train.shape}')
print(f'shape X_numpy_test: {X_numpy_test.shape}')
print(f'shape y_numpy_train: {y_numpy_train.shape}')
print(f'shape y_numpy_test: {y_numpy_test.shape}')
print("-----------------------")
print(f'shape X_train: {X_train.shape}')
print(f'shape X_test: {X_test.shape}')
print(f'shape y_train: {y_train.shape}')
print(f'shape y_test: {y_test.shape}')
print("-----------------------")

['unique_id', 'filename', 'sparql_file', 'profile', 'limit', 'ql_rt_msec', 'ql_rt_clocks', 'ql_rnd_rows', 'ql_seq_rows', 'ql_same_seg', 'ql_same_page', 'ql_disk_reads', 'ql_spec_disk_reads', 'ql_cl_wait_clocks', 'ql_c_msec', 'ql_c_disk', 'ql_c_clocks', 'ql_cl_messages', 'ql_c_cl_wait', 'matrix_format', 'binary_tree']
shape df_raw: (3711, 21)
shape df_clean: (2253, 21)
shape x: (2253, 5940)
shape x_trans: (5940, 2253)
Sigo vivo xd
Sigo vivo xd
Sigo vivo xd
Sigo vivo xd
Sigo vivo xd
Sigo vivo xd
shape df_clean + caracteristicas de vectores: (2253, 5961)
-----------------------
shape X_numpy_train: (1830, 5954)
shape X_numpy_test: (423, 5954)
shape y_numpy_train: (1830,)
shape y_numpy_test: (423,)
-----------------------
shape X_train: torch.Size([1830, 5954])
shape X_test: torch.Size([423, 5954])
shape y_train: torch.Size([1830, 1])
shape y_test: torch.Size([423, 1])
-----------------------


## Parametros

In [7]:
n_samples_train, n_features_train = X_train.shape
n_samples_test, n_features_test = X_test.shape
learning_rate = 0.01
hidden_size = 100000
input_size = n_features_train 
print(f'n_samples_train: {n_samples_train}')
print(f'n_features_train: {n_features_train}')
print(f'n_samples_test: {n_samples_test}')
print(f'n_features_test: {n_features_test}')
print(f'learning_rate: {learning_rate}')
print(f'hidden_size: {hidden_size}')
print(f'input_size: {input_size}')


n_samples_train: 1830
n_features_train: 5954
n_samples_test: 423
n_features_test: 5954
learning_rate: 0.01
hidden_size: 100000
input_size: 5954


## Modelo

In [11]:
class NeuralNet(nn.Module):
    def __init__(self,input_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, 30000)
        self.l2 = nn.Linear(30000, 3000)
        self.l3 = nn.Linear(3000, 300)
        self.l4 = nn.Linear(300, 30)
        self.l5 = nn.Linear(30, 1)
        
    # Se aplica el modelo construido arriba en el forward    
    def forward(self,x):
        x=self.l1(x)
        x=F.relu(x)
        x=self.l2(x)
        x=F.relu(x)
        x=self.l3(x)
        x=F.relu(x)
        x=self.l4(x)
        x=F.relu(x)
        x=self.l5(x)
        return x
        
# Definimos el modelo con la clase
model = NeuralNet(input_size).to(device)

## Loss y Optimizer

In [12]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

## Training Loop

In [13]:
for epoch in range(10000):
    y_hat = model(X_train)
    loss = criterion(y_hat,y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1)%100==0:
        print(f'epoch: {epoch+1}, loss = {loss.item()}')

epoch: 100, loss = 584738.25
epoch: 200, loss = 580652.8125
epoch: 300, loss = 559072.6875
epoch: 400, loss = 545991.6875


KeyboardInterrupt: 

In [ ]:
for epoch in range(10000):
    y_hat = model(X_train)
    loss = criterion(y_hat,y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1)%100==0:
        print(f'epoch: {epoch+1}, loss = {loss.item()}')